In [2]:
import sys
!{sys.executable} -m pip install censusdata
import requests
!pip install psycopg2
import pandas as pd
import re
import psycopg2
from sqlalchemy import create_engine
from io import StringIO



In [3]:
HOST = "https://api.census.gov/data"
year = "2019"
data = "acs/acs5"
url = "/".join([HOST, year, data])




In [4]:
specs = {}
get_var = ["NAME", "B01001_002E", "B01001_026E", 'B19013_001E', 'B01002_001E', 'B01002_002E', 'B01002_003E']
specs["get"] = ",".join(get_var)
specs["for"] = "block%20group:*"
specs["in"] = "state:36%20county:*;tract:*"

api_pull = url + "?get=" + specs["get"] + "&for=" + specs["for"] + "&in=" + specs["in"]



col_names = ["Locale", "Male_Population", "Female_Population","Median_Income", "Median_Age", "Median_Age_Male", "Median_Age_Female","Census_State_Num", "Census_County_Num", "Tract", "Block_Group"]
r = requests.get(api_pull)


df = pd.DataFrame(columns = col_names, data = r.json()[1:])


locale_list = df['Locale'].tolist()

blockGroups = []
tracts = []
counties = []
states = []


for l in locale_list:
    bg = l.split(',')[0]
    numericBg = re.sub("[^0-9]", "",bg )
    blockGroups.append(numericBg)
    
    tract = l.split(',')[1]
    numericTract = re.sub("[^0-9]", "",tract )
    tracts.append(numericTract)
    
    county = l.split(',')[2]
    county = county.replace('County', '')
    counties.append(county)
    
    states.append(l.split(',')[3])
    

df["County"]  = counties  
df["State"] = states
df = df.drop(columns=['Locale'])

correct_order = ["Male_Population", "Female_Population","Median_Income", "Median_Age", "Median_Age_Male", "Median_Age_Female",  "State", "County", "Tract", "Block_Group", "Census_State_Num", "Census_County_Num"]

df = df.reindex(columns=correct_order)


#print(df)



      Male_Population Female_Population Median_Income Median_Age  \
0                 364               396         44821       48.4   
1                 547               728         30361       30.6   
2                 994              1194         24271       24.8   
3                  86               422         43220       27.4   
4                 736               614         24646       31.7   
...               ...               ...           ...        ...   
15458             338               231         60625       53.3   
15459             805               845         65054       39.8   
15460             446               437         72679       63.7   
15461             420               719         36250       53.8   
15462             420               336         71213       34.7   

      Median_Age_Male Median_Age_Female      State    County   Tract  \
0                35.6              54.4   New York   Albany   000100   
1                24.3              38.1

In [5]:
username = "mlpp_student"
password = "CARE-horse-most"
dbhost = "acs-db.mlpolicylab.dssg.io" 
port = 5432 
dbname = "acs_data_loading" 
    



conn = psycopg2.connect(
   database="acs_data_loading", user='mlpp_student', password='CARE-horse-most', host='acs-db.mlpolicylab.dssg.io', port= '5432'
)



# Create table statement

table_create_sql = '''
CREATE TABLE  ksoto_acs_data6  (             Id                  bigInt,
                                            Male_Population     bigint,
                                            Female_Population   bigint,
                                            Median_Income       double precision,
                                            Median_Age          double precision,
                                            Median_Age_Male     double precision,
                                            Median_Age_Female   double precision,
                                            State               varchar(128),
                                            County              varchar(128),
                                            Tract               varchar(128),
                                            Block_Group         varchar(128),
                                            Census_State_Num    varchar(128),
                                            Census_County_Num   varchar(128))
'''




cursor = conn.cursor()
cursor.execute(table_create_sql)


conn.commit()




Table dropped... 


In [13]:
#print(df.columns)


sio = StringIO()
sio.write(df.to_csv(index=True, header=None))
sio.seek(0)

cur = conn.cursor()


cur.copy_from(sio, "ksoto_acs_data6", sep =',')
conn.commit()


cur.execute('''SELECT * from ksoto_acs_data6''')

#Fetching 1st row from the table
result = cur.fetchone();
print(result)



(0, 364, 396, 44821.0, 48.4, 35.6, 54.4, ' New York', ' Albany ', '000100', '1', '36', '001')
